<a href="https://colab.research.google.com/github/irissun9602/DeepLearningProject/blob/master/Sick_Leaves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('./mount')

Drive already mounted at ./mount; to attempt to forcibly remount, call drive.mount("./mount", force_remount=True).


*이탤릭체 텍스트*

In [ ]:
import os
import shutil


original_dataset_dir = './mount/My Drive/Colab Notebooks/sick_leaves/dataset'
classes_list = os.listdir(original_dataset_dir)

base_dir = './mount/My Drive/Colab Notebooks/sick_leaves/splitted'

In [ ]:
train_dir = os.path.join(base_dir, 'train')
#os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
#os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
#os.mkdir(test_dir)

#for cls in classes_list:
#    os.mkdir(os.path.join(train_dir, cls))
#    os.mkdir(os.path.join(validation_dir, cls))
#    os.mkdir(os.path.join(test_dir, cls))

In [ ]:
classes_list = os.listdir(original_dataset_dir)

In [ ]:
import math

for cls in classes_list:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)

    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)

    train_fnames = fnames[:train_size]
    print("Train size(",cls,"): ", len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst)

    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print("Validation size(",cls,"): ", len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)

    test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]

    print("Test size(",cls,"): ", len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)

Train size( Potato___healthy ):  91
Validation size( Potato___healthy ):  30
Test size( Potato___healthy ):  30
Train size( Pepper,_bell___Bacterial_spot ):  598
Validation size( Pepper,_bell___Bacterial_spot ):  199
Test size( Pepper,_bell___Bacterial_spot ):  199
Train size( Corn___healthy ):  697
Validation size( Corn___healthy ):  232
Test size( Corn___healthy ):  232
Train size( Tomato___Septoria_leaf_spot ):  1062
Validation size( Tomato___Septoria_leaf_spot ):  354
Test size( Tomato___Septoria_leaf_spot ):  354
Train size( Apple___Black_rot ):  372
Validation size( Apple___Black_rot ):  124
Test size( Apple___Black_rot ):  124
Train size( Peach___Bacterial_spot ):  1378
Validation size( Peach___Bacterial_spot ):  459
Test size( Peach___Bacterial_spot ):  459
Train size( Apple___healthy ):  987
Validation size( Apple___healthy ):  329
Test size( Apple___healthy ):  329
Train size( Strawberry___Leaf_scorch ):  665
Validation size( Strawberry___Leaf_scorch ):  221
Test size( Strawb

In [ ]:
# 베이스라인 모델 학습을 위한 준비
import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

BATCH_SIZE = 256
EPOCH =30

import torchvision.transforms as trasnforms
from torchvision.datasets import ImageFolder
transform_base = trasnforms.Compose([trasnforms.Resize((64,64)), trasnforms.ToTensor()])
train_dataset = ImageFolder(root='./mount/My Drive/Colab Notebooks/sick_leaves/splitted/train',
                            transform=transform_base)
val_dataset = ImageFolder(root='./mount/My Drive/Colab Notebooks/sick_leaves/splitted/val',
                          transform=transform_base)

from torch.utils.data import DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=BATCH_SIZE, shuffle=True,
                                           num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


In [ ]:
# 베이스라인 모델 설계
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as  optim

class Net(nn.Module):


  def __init__(self):

    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3,32,3, padding=1)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(32,64,3, padding=1)
    self.conv3 = nn.Conv2d(64,64,3, padding=1)

    self.fc1 = nn.Linear(4096, 512)
    self.fc2 = nn.Linear(512, 33)


  def forward(self, x):

    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)
    x = F.dropout(x, p=0.25, training=self.training)

    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)
    x = F.dropout(x, p=0.25, training=self.training)

    x = self.conv3(x)
    x = F.relu(x)
    x = self.pool(x)
    x = F.dropout(x, p=0.25, training=self.training)

    x = x.view(-1, 4096)
    x = self.fc1(x)
    x = F.relu(x)
    x = F.dropout(x, p=0.5, training=self.training)
    x = self.fc2(x)

    return F.log_softmax(x, dim=1)

model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)


In [ ]:
# 모델 학습을 위한 함수
def train(model, train_loader, optimizer):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

In [ ]:
# 모델 평가를 위한 함수
def evaluate(model, tes_loader):
  model.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(DEVICE), target.to(DEVICE)
      output = model(data)

      test_loss += F.cross_entropy(output,
                                   target, reduction='sum').item()

      pred = output.max(1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accurancy

In [ ]:
#모델 학습 실행하기
import time
import copy

def train_baseline(model, train_loader, val_loader,
                   optimizer, num_epochs =30):
  best_acc = 0.0
  best_model_wts = copy.deepcopy(model.state_dict())

  for epoch in range(1, num_epochs + 1):
    since = time.time()
    train(model, train_loader, optimizer)
    train_loss, train_acc = evaluate(model, train_loader)
    val_loss, val_acc = evaluate(model, val_loader)
    if val_acc > best_acc:
      best_acc = val_acc
      best_model_wts = copy.deepcop(model.state_dict())

    time_elapsed = time.tiem() - since
    print('------------------epoch {} ---------------'.format(epoch))

    print('train Loss : {:.4f}, Accuracy: {:.2f}%'
    .format(train_loss, train_acc))
    print('val Loss : {:.4f}, Accuracy: {:.2f}%'
    .format(val_loss, val_acc))
    print('Completed in {:.0f}m {:.0f}s'
    .format(time_elapsed // 60, time_elapsed % 60))
  model.load_state_dict(best_model_wts)
  return model

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)
torch.save(base, 'baseline.pt')



KeyboardInterrupt: ignored